## Important Dependencies with their required versions


In [ ]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb
# !pip install text_generation

## Imports and Setup and Initializing Agent

In [1]:
# # to auto reload instead of having to restart each time
%load_ext autoreload
%autoreload 2

In [1]:
# from Imports import *
try:
    from Configuration import *
    from TestGenerator import *
    from TestFix import *
    from DecisionMaker import *
    from BugFix import *

    # llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"],repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1')
    testGenerator = TestGenerator(GenUnitTestChain, db, HEval_JsonObj, globals())
    testRegenerator = TestFix(UnitTestFeedbackChain,True,globals())
    judgeGenerator = DecisionMaker(judgeChain,globals())
    bugFixGenerator = BugFix(bugFixChain,True,globals())
except Exception as e:
    print(e)
    exit(-1)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.
C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


## Generate Unit Tests from Code

In [ ]:
testGenerator.generate()

## Judge

In [2]:
judgeGenerator.generate()

File JudgeOutput/JudgmentLogs.json created successfully!
Running Test Case  0
generatedJudgement:  <s> [INST] You are python unit tester judge. Given the following python code, its description, a test case that produce an error and the error message.
You are going to follow the criteria that I give to you.
You follow my rules and orders and if you do not know the answer, don't make things UP!
Criteria:
1. Understand the description and the python code under test.
2. Interpret the test case that produces an error and the accompanying error message.
3. Identify wether the error produced when excuting the test case is due to a bug in the python code under test or in the test case itself.
4. Provide a clear explanation for your decision.
Follow the provided criteria to make an informed decision.

Method under test:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(

In [ ]:
judgeOut=1
if judgeOut==1:
    pass
else:
    testRegenerator.generate()